In [1]:
from neo4j import GraphDatabase
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import networkx as nx
from tensorflow import keras
from datetime import datetime

In [2]:
m = 2
xs = [i * m for i in range(50)]
c = 50

ms = [m for _ in range(50)]
cs = [c for _ in range(50)]
ys = [m * x + c for x in xs]

data = {
    'ms': ms,
    'xs': xs,
    'cs': cs,
    'ys': ys
}

df = pd.DataFrame(data=data)
df.head(5)

,ms,xs,cs,ys
0,2,0,50,50
1,2,2,50,54
2,2,4,50,58
3,2,6,50,62
4,2,8,50,66


In [3]:
X_train = df[['ms', 'xs', 'cs']]
y_train = df['ys']

In [4]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(units=3, input_shape=[3]))
model.add(tf.keras.layers.Dense(units=3, input_shape=[3]))
model.add(tf.keras.layers.Dense(units=1))
model.summary()
model.compile(optimizer=tf.keras.optimizers.Adam(0.5), loss='mean_squared_error')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 3)                 12        
                                                                 
 dense_1 (Dense)             (None, 3)                 12        
                                                                 
 dense_2 (Dense)             (None, 1)                 4         
                                                                 
Total params: 28
Trainable params: 28
Non-trainable params: 0
_________________________________________________________________


In [ ]:
epochs_hist = model.fit(X_train, y_train, epochs=500)

In [12]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [17]:
conn = Neo4jConnection(uri="neo4j://localhost:7687", user="neo4j", pwd="password")

In [167]:
weights = [model.layers[i].get_weights()[0].tolist() for i in range(len(model.layers))]
biases = [model.layers[i].bias.numpy().tolist() for i in range(len(model.layers))]
output_w = weights.pop()
output_b = biases.pop()
print(weights)
print(output_w)
print(len(output_w))
print(biases)
print(output_b)

[[[-0.44424766302108765, -0.14649514853954315, 1.4081895351409912], [0.7082571387290955, -0.22568680346012115, 1.289670705795288], [-0.032005373388528824, -0.26664653420448303, 0.6969856023788452]], [[0.462401419878006, -0.35733285546302795, 0.9182608723640442], [3.4768314361572266, -2.2673356533050537, 0.18476609885692596], [0.598547101020813, -0.5195207595825195, 1.5608936548233032]]]
[[-0.08561587333679199], [-0.02823176234960556], [0.7687111496925354]]
3
[[0.09486032277345657, 0.6421387195587158, 0.8253923654556274], [-0.40326762199401855, 1.972714900970459, 2.245237350463867]]
[3.948150634765625]


In [168]:
def input_nodes():
    for i in range(3):
        query = f"CREATE (i: Input {{name: 'input_{i}'}})"
        conn.query(query, db="nndb")

In [169]:
def hidden_layer_nodes():
    for i in range(len(biases)):
        for j in range(len(biases[i])):
            query = f"CREATE (h: HiddenLayer{i} {{bias: {biases[i][j]}, name: 'hidden_layer_{i}_node_{j}'}})"
            conn.query(query, db="nndb")

In [170]:
def output_nodes():
    query = f"CREATE (o: Output {{bias: {output_b[0]}, name: 'output_node'}})"
    conn.query(query, db="nndb")

In [171]:
def input_weights():
    in_weights = weights[0]
    for i in range(len(in_weights)):
        for j in range(len(in_weights[i])):
            query = f"""
                MATCH
                    (i: Input),
                    (h: HiddenLayer0)
                WHERE i.name = 'input_{i}'
                AND h.name = 'hidden_layer_0_node_{j}'
                CREATE (i)-[w: Weight {{weight: {in_weights[i][j]}}}]->(h)
                RETURN i, h
            """
            conn.query(query, db="nndb")

In [172]:
def output_weights():
    last_hidden_layer = len(biases) - 1
    for i in range(len(output_w)):
        query = f"""
            MATCH 
                (h: HiddenLayer{last_hidden_layer}),
                (o: Output)
            WHERE h.name = 'hidden_layer_{last_hidden_layer}_node_{i}'
            CREATE (h)-[w: Weight {{weight: {output_w[i][0]}}}]->(o)
        """
        conn.query(query, db="nndb")

In [191]:
def hidden_layer_weights():
    for i in range(len(weights)):
        if i == 0:
            continue
        for j in range(len(weights[i])):
            for k in range(len(weights[i][j])):
                query = f"""
                    MATCH
                        (h0: HiddenLayer{i - 1}),
                        (h1: HiddenLayer{i})
                    WHERE h0.name = 'hidden_layer_{i - 1}_node_{j}'
                    AND h1.name = 'hidden_layer_{i}_node_{k}'
                    CREATE (h0)-[w: Weight {{weight: {weights[i - 1][j][k]}}}]->(h1)
                """
                conn.query(query, db="nndb") 
            

In [192]:
conn.query("CREATE OR REPLACE DATABASE nndb")
input_nodes()
hidden_layer_nodes()
output_nodes()
input_weights()
hidden_layer_weights()
output_weights()

hidden_layer_0_node_0
hidden_layer_1_node_0
hidden_layer_0_node_0
hidden_layer_1_node_1
hidden_layer_0_node_0
hidden_layer_1_node_2
hidden_layer_0_node_1
hidden_layer_1_node_0
hidden_layer_0_node_1
hidden_layer_1_node_1
hidden_layer_0_node_1
hidden_layer_1_node_2
hidden_layer_0_node_2
hidden_layer_1_node_0
hidden_layer_0_node_2
hidden_layer_1_node_1
hidden_layer_0_node_2
hidden_layer_1_node_2
